# Multi-layer Perceptron (MLP)

외부 파일 가져오기 & requirements 설치

In [ ]:
!pwd

mount하기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import sys
sys.path.append('/content/drive/MyDrive/#fastcampus')

In [ ]:
!ls

!pip install -r '/content/drive/MyDrive/#fastcampus/requirements.txt'

https://pytorch.org/vision/stable/transforms.html

In [ ]:
import numpy as np
from tqdm import tqdm   # 진행율
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F # relu 등 함수 모음
from torch import optim
from torch.utils.tensorboard import SummaryWriter

from torchvision.datasets import FashionMNIST
from torchvision import transforms    # feature engineering 전처리 작업 효율적으로 할 수 있게 도와줌
from torch.utils.data import random_split  # dataset split

왼쪽 파일 부분 보면 'data' 라는 폴더가 새로 생겼고 FahionMNIST data download 된 것 볼 수 있음</br>
'raw' 파일 안에 압축 파일들 생성됨<br>
https://pytorch.org/vision/stable/generated/torchvision.datasets.FashionMNIST.html
- 링크 들어가서 보면 getitem으로 정의되어 있는 데이터라는 것 알 수 있음
- 이런 dataset은 iterator가 아니기 때문에 list 같이 index를 통해 특정 value 바로 접근 가능
- iterator dataset은 next 함수 이용해서 접근 가능

In [ ]:
data_root = os.path.join(os.getcwd(), 'data')

# 전처리 부분 (preprocessing) & 데이터 셀 출력
transform = transforms.Compose(
    [
     transforms.ToTensor(),
     transforms.Normalize([0.5], [0.5]), # mean, std
    ]
)
fashion_mnist_dataset = FashionMNIST(data_root, download=True, train=True, transform=transform)

list처럼 특정 이미지 바로 접근하기
- normalize해서 다운받았기 때문에 어느 정도 normalize된 데이터

In [ ]:
fashion_mnist_dataset[0][0]

첫 번째 데이터의 label

In [ ]:
fashion_mnist_dataset[0][1]

dataset split

In [ ]:
dset = random_split(fashion_mnist_dataset, [int(len(fashion_mnist_dataset)*0.7), int(len(fashion_mnist_dataset)*0.3)])

In [ ]:
dset[0]

data_utils
- 강사가 미리 만들어놓은 코드

In [ ]:
from data_utils import dataset_split

In [ ]:
datasets = dataset_split(fashion_mnist_dataset, split=[0.9, 0.1])
print(datasets)

dataloader 정의

In [ ]:
train_dataset = datasets["train"]
val_dataset = datasets["val"]

# batch 단위로 데이터를 묶을 예정
train_batch_size = 100
val_batch_size = 10

# num_workers : 병렬 processing
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=train_batch_size, shuffle=True, num_workers=1
)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=val_batch_size, shuffle=False, num_workers=1
)

In [ ]:
for sample_batch in train_dataloader:
    print(sample_batch)
    # 그림과 label 각각에 대한 shape 확인
    print(sample_batch[0].shape, sample_batch[1].shape)
    break

## 모델 정의 (Multi-layer Perceptron) (MLP)

pytorch model은 보통 class로 정의
- nn과 같은 module을 꼭 import 해야 함 ★ : GPU, CPU 간 꼬이는 것 방지

In [ ]:
class MLP(nn.Module):
    # input, hidden layer1/2, output의 차원을 먼저 알아야 함
    def __init__(self, in_dim: int, h1_dim: int, h2_dim: int, out_dim: int):
        # nn.Module 가져오기
        super().__init__()
        # MLP layer를 각각 정의할 때 제일 처음 linear layer를 지나야 함
        self.linear1 = nn.Linear(in_dim, h1_dim)
        self.linear2 = nn.Linear(h1_dim, h2_dim)
        self.linear3 = nn.Linear(h2_dim, out_dim)
        self.relu = F.relu
    
    def forward(self, input):
        # 위에서 input shape : torch.Size([100, 1, 28, 28]) → 1×28×28
        x = torch.flatten(input, start_dim=1)
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        out = self.linear3(x)
        # output에 sigmoid(binary)나 softmax(multi) 씌워서 출력해도 되고 출력 후 나중에 작업해도 됨
        # out = F.softmax(out)
        return out

## 모델 선언 및 손실 함수, 최적화(optimizer) 정의, Tensorboard Logger 정의

아래 셀 값들 조절해서 다른 모델 만들어 볼 수 있음

In [ ]:
# define model
# input shape : 28*28
# hidden layer : 임의로 일단 128, 64로 정함 -> 수정 가능 (0.5배, 2배, 3배 등)
# output : label 0~9까지 있으니 10
model = MLP(28*28, 128, 64, 10)

# define loss
loss_function = nn.CrossEntropyLoss()

# define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
max_epoch = 15

# define tensorboard logger
writer = SummaryWriter()
log_interval = 100

아래 코드 돌릴 때는 새로 생성되었던 runs 파일 삭제하고 다시 돌려야 함

In [ ]:
%load_ext tensorboard
# runs라는 폴더를 만들고 그 안에 logging을 쌓을 예정
%tensorboard --logdir runs/

train_step = 0
for epoch in range(1, max_epoch+1):
    
    # valid step
    # ★TIP : train step보다 먼저 해 보면 랜덤 input에 대해 얼마나 막장인지 미리 결과 확인해 볼 수 있으니 좋음
    # 단, validation data에 대해서는 optimizer를 하면 안 됨 -> cheating이니까
    # pytorch에서는 'torch.no_grad()'만 해도 optimizer 안 한 것으로 바꿔줌
    with torch.no_grad():
        val_loss = 0.0
        val_corrects = 0

        # enumerate 안에 tqdm 해 주면 결과를 더 예쁘게 볼 수 있음
        # https://tqdm.github.io/docs/tqdm/
        # - position : Specify the line offset to print this bar (starting from 0) -> Useful to manage multiple bars at once
        #              ipynb은 자꾸 밑으로 내려가는데(?) 'position=0'으로 해야 가만히 있음 
        # - leave=True : keeps all traces of the progressbar upon termination of iteration
        # - desc : 진행바 앞에 text 출력
        for val_batch_idx, (val_images, val_labels) in enumerate(
            tqdm(val_dataloader, position=0, leave=True, desc="validation")
            ):
            # forward
            val_outputs = model(val_images)
            _, val_preds = torch.max(val_outputs, 1)

            # loss & acc
            # val_outputs.shape[0] : batch size
            val_loss += loss_function(val_outputs, val_labels) / val_outputs.shape[0]
            val_corrects += torch.sum(val_preds == val_labels.data) / val_outputs.shape[0]
            
    # valid step logging
    val_epoch_loss = val_loss / len(val_dataloader)
    val_epoch_acc = val_corrects / len(val_dataloader)

    print(
        f"{epoch} epoch, {train_step} step: val_loss: {val_epoch_loss}, val_acc: {val_epoch_acc}"
    )

    # tensorboard에 write
    # val_step 말고 train_step으로 해야 서로 같은 시점에서 비교하기 좋음
    writer.add_scalar('Loss/val', val_epoch_loss, train_step)
    writer.add_scalar('Acc/val', val_epoch_acc, train_step)
    writer.add_images('Images/val', val_images, train_step)

    # ===============================================================

    # train step
    for batch_idx, (images, labels) in enumerate(
        tqdm(train_dataloader, position=0, leave=True, desc='training')
        ):

        current_loss = 0.0
        current_corrects = 0  # 몇 개나 맞췄는지
        
        # Forward ===================================================
        # get predictions
        outputs = model(images)
        _, preds = torch.max(outputs, 1)  # 1 : dimension
        # print(outputs)
        # print(preds)

        # get loss
        # Cross Entropy : loss_function(input, target)
        loss = loss_function(outputs, labels)
        
        # Backpropagation ===========================================
        # optimizer 초기화 (zero화) : garbage 값이 있을 수도 있으니 처리해주기
        optimizer.zero_grad()

        # Perform backward pass
        # Chain rule 자동 계산해 줌
        loss.backward()

        # Perform Optimization
        optimizer.step()

        current_loss += loss.item()
        current_corrects += torch.sum(preds == labels.data)

        # 일정 이상 돌면 = 100번(log_interval)마다 정확도 평균 계산
        if train_step % log_interval == 0:
            train_loss = current_loss / log_interval
            train_acc = current_corrects / log_interval

            print(
                f'{train_step}: train_loss: {train_loss}, train_acc: {train_acc}'
            )

            # tensorboard에 write
            writer.add_scalar('Loss/train', train_loss, train_step)
            writer.add_scalar('Acc/train', train_acc, train_step)
            writer.add_images('Images/train', images, train_step)
            writer.add_graph(model, images)

            # loss 초기화
            current_loss = 0
            current_corrects= 0

        train_step += 1
        # break
    # break

## save model

- ckpt : checkpoint

In [ ]:
os.makedirs('./logs/models', exist_ok=True)
torch.save(model, './logs/models/mlp.ckpt')

load model

- eval(expression) : expression을 문자열로 받아 실행

In [ ]:
loaded_model = torch.load('./logs/models/mlp.ckpt')
loaded_model.eval()
print(loaded_model)  # 잘 load 되었는지 확인

## softmax

In [ ]:
def softmax(x, axis=0):
    'numpy softmax'
    max = np.max(x, axis=axis, keepdims=True)
    e_x = np.exp(x - max)
    sum = np.sum(e_x, axis=axis, keepdims=True)
    f_x = e_x / sum
    return f_x

## test model

torch.tensor
- It has an additional "computational graph" layer
- To convert a `torch.tensor` to `np.ndarray`, you must explicitly remove the computational graph of the tensor using the ★ `detach()` command

★ 분석을 할 때는 마지막에 다 numpy로 바꿔줘야 scikit learn 등 여러 가지 tool에 쉽고 유연하게 적용이 가능하기 때문에 아래와 같이 대부분 마지막에는 numpy로 바꿔주고 끝냄

In [ ]:
test_batch_size = 100
test_dataset = FashionMNIST(data_root, download=True, train=False, transform=transforms.ToTensor())
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False, num_workers=1)

test_labels_list = []
test_preds_list = []
test_outputs_list = []

for i, (test_images, test_labels) in enumerate(tqdm(test_dataloader, position=0, leave=True, desc='testing')):
    # forward
    test_outputs = loaded_model(test_images)
    _, test_preds = torch.max(test_outputs, 1)

    final_outs = softmax(test_outputs.detach().numpy(), axis=1)
    test_outputs_list.extend(final_outs)
    test_preds_list.extend(test_preds.detach().numpy())
    test_labels_list.extend(test_labels.detach().numpy())

# accuracy를 구하기 위해 numpy array로 변경
test_preds_list = np.array(test_preds_list)
test_labels_list = np.array(test_labels_list)

print(f'\nacc: {np.mean(test_preds_list == test_labels_list)*100}%')

## ROC curve

- https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

fpr = {}
tpr = {}
thresh = {}  # threshold
n_class = 10

for i in range(n_class):
    # [:, i] : batch size는 유지하고 각 class마다 확인
    fpr[i], tpr[i], thresh[i] = roc_curve(test_labels_list, np.array(test_outputs_list)[:, i], pos_label=i)

전체 test case에 대한 값

In [ ]:
print(fpr)

In [ ]:
for i in range(n_class):
    plt.plot(fpr[i], tpr[i], linestyle='--', label=f'Class {i} vs Rest')
plt.title('Multi-class ROC Curve')
plt.xlabel('False Position Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='best')
plt.show()

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html

- ovo : Stands for One-vs-one. Computes the average AUC of all possible pairwise combinations of classes.
- macro : Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.

In [ ]:
print('auc_score ', roc_auc_score(test_labels_list, test_outputs_list, multi_class='ovo', average='macro'))